<a href="https://colab.research.google.com/github/gemstuition/END_Program/blob/main/session7_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = pd.read_csv('datasetSentences.txt',sep='\t')
data.head()
#print(data.shape)

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [2]:
import pandas as pd
labels = pd.read_csv('sentiment_labels.txt',sep='|')
labels.head()
#labels.shape

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [5]:
for i,row in labels.iterrows():
  x = row['sentiment values']
  x = int(x*25)
  labels.at[i,'sentiment values'] = x

In [8]:
df = pd.concat([data, labels], axis=1)

In [9]:
for i,rows in df.iterrows():
  x = rows['sentiment values']
  if x <= 5.0:
    df['sentiment values'].iloc[i] = 0 #'very negative'
  elif x <= 10.0 and x > 5.0:
    df['sentiment values'].iloc[i] = 1 #'negative'
  elif x <= 15.0 and x > 10.0:
    df['sentiment values'].iloc[i] = 2 #'neutral'
  elif x <= 20.0 and x > 15.0:
    df['sentiment values'].iloc[i] = 3 #'positive'
  elif x <= 25.0:
    df['sentiment values'].iloc[i] = 4 #'very positive'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
df.shape

(81134, 4)

In [13]:
df.head()

,sentence,phrase ids,sentiment values
0,The Rock is destined to be the 21st Century 's...,0,2.0
1,The gorgeously elaborate continuation of `` Th...,1,2.0
2,Effective but too-tepid biopic,2,2.0
3,If you sometimes like to go to the movies to h...,3,2.0
4,"Emerges as something rare , an issue movie tha...",4,1.0


In [15]:

df.dropna(subset = ["sentence"], inplace=True)

In [16]:
df.head()

,sentence,phrase ids,sentiment values
0,The Rock is destined to be the 21st Century 's...,0,2.0
1,The gorgeously elaborate continuation of `` Th...,1,2.0
2,Effective but too-tepid biopic,2,2.0
3,If you sometimes like to go to the movies to h...,3,2.0
4,"Emerges as something rare , an issue movie tha...",4,1.0


In [18]:
df.drop('phrase ids',axis=1,inplace=True)

In [19]:
df.head()

,sentence,sentiment values
0,The Rock is destined to be the 21st Century 's...,2.0
1,The gorgeously elaborate continuation of `` Th...,2.0
2,Effective but too-tepid biopic,2.0
3,If you sometimes like to go to the movies to h...,2.0
4,"Emerges as something rare , an issue movie tha...",1.0


In [20]:
df.columns = ['tweets', 'labels']

In [21]:
df.head()

,tweets,labels
0,The Rock is destined to be the 21st Century 's...,2.0
1,The gorgeously elaborate continuation of `` Th...,2.0
2,Effective but too-tepid biopic,2.0
3,If you sometimes like to go to the movies to h...,2.0
4,"Emerges as something rare , an issue movie tha...",1.0


In [22]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [23]:

Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [24]:
fields = [('tweets', Tweet),('labels',Label)]

In [25]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])]

In [26]:
# Creating dataset

twitterDataset = data.Dataset(example, fields)

In [27]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [28]:

(len(train), len(valid))

(10077, 1778)

In [29]:
x_lst = []
y_lst = []
for i in range(len(train.examples)):
  xx = vars(train.examples[i])['tweets']
  xxx = ' '.join(xx)
  x_lst.append(xxx)
  yy = vars(train.examples[i])['labels']
  y_lst.append(yy)

In [30]:
train_df = pd.DataFrame(
    {'tweets': x_lst,
     'labels': y_lst
    })

In [31]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [32]:

print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18834
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 9498), (',', 8475), ('the', 7175), ('and', 5253), ('a', 5194), ('of', 5183), ('to', 3608), ('-', 3226), ("'s", 3003), ('is', 2979)]
Labels :  defaultdict(<function _default_unk_index at 0x7fa487a63268>, {2.0: 0, 1.0: 1, 3.0: 2, 0.0: 3, 4.0: 4})


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

In [36]:

import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

In [38]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [39]:

# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.5

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [40]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(18834, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,892,305 trainable parameters


In [41]:

import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [42]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [44]:

N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.407 | Train Acc: 52.27%
	 Val. Loss: 1.381 |  Val. Acc: 52.61% 

	Train Loss: 1.374 | Train Acc: 53.25%
	 Val. Loss: 1.379 |  Val. Acc: 52.61% 

	Train Loss: 1.373 | Train Acc: 53.25%
	 Val. Loss: 1.379 |  Val. Acc: 52.61% 

	Train Loss: 1.372 | Train Acc: 53.26%
	 Val. Loss: 1.379 |  Val. Acc: 52.61% 

	Train Loss: 1.372 | Train Acc: 53.31%
	 Val. Loss: 1.379 |  Val. Acc: 52.61% 

	Train Loss: 1.372 | Train Acc: 53.34%
	 Val. Loss: 1.379 |  Val. Acc: 52.67% 

	Train Loss: 1.372 | Train Acc: 53.37%
	 Val. Loss: 1.379 |  Val. Acc: 52.67% 

	Train Loss: 1.371 | Train Acc: 53.50%
	 Val. Loss: 1.379 |  Val. Acc: 52.61% 

	Train Loss: 1.370 | Train Acc: 53.55%
	 Val. Loss: 1.379 |  Val. Acc: 52.61% 

	Train Loss: 1.369 | Train Acc: 53.63%
	 Val. Loss: 1.379 |  Val. Acc: 52.55% 



In [ ]:

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "very negative", 1:"negative", 2:"neutral",3:"positive",4:"very positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

In [ ]:

def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [ ]:
sentence = train_df.tweets.values
print(len(random_swap(sentence,n=2)))
print(len(sentence))

In [ ]:
random_deletion(sentence, p=0.5)

In [ ]:
import random
import googletrans_new as googletrans.translate
#from googletrans_new import googletrans.Translator

translator = Translator()
sentence = ['The dog slept on the rug']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)